In [4]:
import pdfplumber
import pandas as pd
import re

# Step 1: Extract tables from PDF and save them into an Excel file
def extract_and_clean_pdf_tables(pdf_path, excel_path):
    # Open the PDF
    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []
        
        # Loop through each page in the PDF
        for page_num, page in enumerate(pdf.pages):
            # Extract tables from the page
            tables = page.extract_tables()
            
            # If tables are found on the page
            if tables:
                for table in tables:
                    # Convert the table into a pandas DataFrame
                    df = pd.DataFrame(table)
                    
                    # Clean the "Amount" column to remove the $ sign
                    # Assuming the first row contains headers
                    df.columns = df.iloc[0]  # Set first row as header
                    df = df.drop(0).reset_index(drop=True)  # Drop the header row
                    
                    # Check if 'Amount' column exists, and clean it
                    if 'Amount' in df.columns:
                        # Clean the 'Amount' column by removing '$' and converting to float
                        df['Amount'] = df['Amount'].apply(lambda x: re.sub(r'[^\d.-]', '', str(x)) if pd.notnull(x) else x).astype(float)
                    
                    # Append the cleaned DataFrame
                    all_tables.append(df)
        
        # Create an Excel writer object
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # Save each cleaned table as a sheet in the Excel file
            for i, table_df in enumerate(all_tables):
                table_df.to_excel(writer, sheet_name=f'Table_{i+1}', index=False)

# Step 2: Reconcile the bank statement with the company ledger and save the results
def reconcile_bank_statements(bank_sheet_path, company_sheet_path, reconciliation_result_path):
    # Load bank statement and company ledger into pandas DataFrames
    bank_df = pd.read_excel(bank_sheet_path)
    company_df = pd.read_excel(company_sheet_path)

    # Merge the two DataFrames based on common identifiers (like Date and Description)
    # Perform a left join to keep all bank transactions and match with company transactions
    merged_df = pd.merge(bank_df, company_df, on=['Date', 'Description'], how='left', suffixes=('_bank', '_company'))

    # Calculate the difference between the Bank Amount and Company Amount
    merged_df['Amount Difference'] = merged_df['Amount_bank'] - merged_df['Amount_company']

    # Separate the matched (reconciled) and unmatched (unreconciled) transactions
    reconciled_df = merged_df[merged_df['Amount Difference'] == 0]
    unreconciled_df = merged_df[merged_df['Amount Difference'] != 0]

    # Write the results to a new Excel file, keeping matched and unmatched transactions in the same sheet
    with pd.ExcelWriter(reconciliation_result_path, engine='openpyxl') as writer:
        # Create a new sheet to show both matched and unmatched data in separate tables
        sheet_name = 'Reconciliation_Result'

        # Write Matched (Reconciled) transactions at the top of the sheet
        reconciled_df.to_excel(writer, sheet_name=sheet_name, startrow=0, index=False)

        # Write a header for the Unmatched (Unreconciled) section
        start_row = len(reconciled_df) + 2  # Leave 2 rows of space between matched and unmatched tables
        writer.sheets[sheet_name].cell(row=start_row, column=1, value="\n Unmatched Transactions")

        # Write Unmatched (Unreconciled) transactions below the header
        unreconciled_df.to_excel(writer, sheet_name=sheet_name, startrow=start_row + 1, index=False)

    print("Bank reconciliation completed and results saved!")

# Paths
pdf_path = 'c:\\Users\\arcks\\Downloads\\bank_statement.pdf'           # PDF containing the bank statement
excel_path = 'c:\\Users\\arcks\\Downloads\\extracted_pdf_data.xlsx'    # Temporary Excel file for extracted tables
company_sheet_path = 'c:\\Users\\arcks\\Downloads\\erp_statement.xlsx' # Company ERP statement
reconciliation_result_path = 'c:\\Users\\arcks\\Downloads\\Bank_Reconciliation.xlsx'  # Reconciliation result file

# Step 1: Extract tables from PDF and save to Excel
extract_and_clean_pdf_tables(pdf_path, excel_path)

# Step 2: Perform bank reconciliation using the extracted tables
reconcile_bank_statements(excel_path, company_sheet_path, reconciliation_result_path)


Bank reconciliation completed and results saved!
